In [1]:
import pandas as pd
import numpy as np

In [35]:
#load the data
# Please change path to the location of CreditModelData1.csv in your computer to preceed further steps!
path = 'CreditModelData1.csv'
data1 = pd.read_csv(path,engine='python')
data1.shape

(10000, 96)

In [3]:
#change dependent variable into 0/1
default = pd.Series(data1['S&P Entity Credit Rating Action [3/3/2008-3/5/2018]'])

default1 = np.where(default=='-', 0, 1)

del data1['S&P Entity Credit Rating Action [3/3/2008-3/5/2018]']


In [4]:
# The original dataset use '-' and 'NM' for missing data, change them into np.NaN 
isUnderLine = data1 == str('-')
isNM = data1 ==str('NM')

data1 = data1.mask(isUnderLine,np.NaN)
data1 = data1.mask(isNM,np.NaN)

In [5]:
data1['default'] = default1

In [6]:
data1 = data1.drop('Company Type',axis=1)

In [7]:
# Delete all columns that contain all missing value
for name in data1.columns[data1.isnull().all()]:
    del data1[name]

In [8]:
# Delete all rows that contain all missing value
numbers = data1.iloc[:,3:-2]
emptyRow = []
for row in range(len(data1)):
    if(numbers.iloc[row].isnull().all()):
        emptyRow.append(row)


data1 = data1.drop(emptyRow,axis=0)

data1.index = np.arange(data1.shape[0])

In [9]:
# Change the data type from string into float in order to do mathematical operation
data1.iloc[:,3:75] = data1.iloc[:,3:75].astype(np.float)

In [10]:
np.sum(data1.isnull()==False, axis=0)

Company Name                                                                                             7086
Exchange:Ticker                                                                                          4532
S&P Entity ID                                                                                             970
EBITDA - Capital IQ [CY 2008] ($USDmm, Historical rate)                                                  3622
EBITDA - Capital IQ [CY 2009] ($USDmm, Historical rate)                                                  3660
EBITDA - Capital IQ [CY 2010] ($USDmm, Historical rate)                                                  3657
EBITDA - Capital IQ [CY 2011] ($USDmm, Historical rate)                                                  3725
EBITDA - Capital IQ [CY 2012] ($USDmm, Historical rate)                                                  3696
EBITDA - Capital IQ [CY 2013] ($USDmm, Historical rate)                                                  3736
EBITDA - C

We found out that all columns for free operating cash flow/debt and current ratio were deleted except two that only containg 438 and 448 values. Since we could not predict missing values for these categories using past year/previous year's record, and we want to keep these columns. We fill these columns with column mean. 

In [11]:
data1['Free Operating Cash Flow/Debt (%), Adj. - Credit Stats Direct [LTM]'] = data1['Free Operating Cash Flow/Debt (%), Adj. - Credit Stats Direct [LTM]'].fillna(data1['Free Operating Cash Flow/Debt (%), Adj. - Credit Stats Direct [LTM]'].mean())

data1['Free Operating Cash Flow/Debt (%), Adj. - Credit Stats Direct [LTM]'] = data1['Current Ratio (%), Adj. - Credit Stats Direct [LTM]'].fillna(data1['Current Ratio (%), Adj. - Credit Stats Direct [LTM]'].mean())

In [12]:
# Divide remaining columns into 9 categories and delete rows that doesn't have any value in any of these 9 categories
columnNumber = [np.arange(3,13),np.arange(13,23),np.arange(23,33),np.arange(33,43),np.arange(43,53),
               np.arange(55,65),np.arange(65,75)]

allNullOneCategory = data1.iloc[:,np.arange(3,13)].isnull().all(axis=1)
for category in columnNumber:
    allNullOneCategory = (allNullOneCategory | data1.iloc[:,category].isnull().all(axis=1))

data = data1[allNullOneCategory!=True]

In [15]:
data.shape

(2509, 77)

So there are 2509 companies left with at least one value in each financial category.

In [16]:
companyName = data['Company Name']

In [17]:
# For each category, we use the mean of avaiable values of each company to fill in missing values
frames = []
for category in columnNumber:
    newData = data.iloc[:,category].T.fillna(np.mean(data.iloc[:,category].T)).T
    frames.append(newData)

frames.append(data.iloc[:,76:])

finalData = pd.concat(frames,axis=1)

finalData.index = companyName

In [21]:
finalData.head(5)

,"EBITDA - Capital IQ [CY 2008] ($USDmm, Historical rate)","EBITDA - Capital IQ [CY 2009] ($USDmm, Historical rate)","EBITDA - Capital IQ [CY 2010] ($USDmm, Historical rate)","EBITDA - Capital IQ [CY 2011] ($USDmm, Historical rate)","EBITDA - Capital IQ [CY 2012] ($USDmm, Historical rate)","EBITDA - Capital IQ [CY 2013] ($USDmm, Historical rate)","EBITDA - Capital IQ [CY 2014] ($USDmm, Historical rate)","EBITDA - Capital IQ [CY 2015] ($USDmm, Historical rate)","EBITDA - Capital IQ [CY 2016] ($USDmm, Historical rate)","EBITDA - Capital IQ [CY 2017] ($USDmm, Historical rate)",...,"Unlevered Free Cash Flow - Capital IQ [CY 2009] ($USDmm, Historical rate)","Unlevered Free Cash Flow - Capital IQ [CY 2010] ($USDmm, Historical rate)","Unlevered Free Cash Flow - Capital IQ [CY 2011] ($USDmm, Historical rate)","Unlevered Free Cash Flow - Capital IQ [CY 2012] ($USDmm, Historical rate)","Unlevered Free Cash Flow - Capital IQ [CY 2013] ($USDmm, Historical rate)","Unlevered Free Cash Flow - Capital IQ [CY 2014] ($USDmm, Historical rate)","Unlevered Free Cash Flow - Capital IQ [CY 2015] ($USDmm, Historical rate)","Unlevered Free Cash Flow - Capital IQ [CY 2016] ($USDmm, Historical rate)","Unlevered Free Cash Flow - Capital IQ [CY 2017] ($USDmm, Historical rate)",default
Company Name,,,,,,,,,,,,,,,,,,,,,
"024 Pharma, Inc. (OTCPK:EEIG)",-0.4970,-0.5250,-0.5420,-0.4400,-0.368,-0.295,-0.398,-0.446,0.624,-0.320778,...,0.994,-1.490,-0.152,-0.121,-0.065,-0.785,0.227,-0.3465,-0.346500,0
"1-800-FLOWERS.COM, Inc. (NasdaqGS:FLWS)",53.2000,31.4000,28.8000,33.5000,42.300,46.400,103.400,79.600,78.900,70.800000,...,47.500,5.470,24.700,24.600,16.500,104.000,21.800,23.9000,77.600000,0
"1347 Property Insurance Holdings, Inc. (NasdaqGM:PIH)",1.7682,1.7682,1.7682,1.7682,0.000,-1.130,5.760,3.670,0.541,1.768200,...,6.218,6.218,6.218,0.000,6.750,13.500,9.080,1.7600,6.218000,0
"2050 Motors, Inc. (OTCPK:ETFM)",-0.3936,-0.3936,-0.3936,-0.3936,0.000,0.000,-0.534,-0.698,-0.736,-0.393600,...,-0.131,-0.131,-0.131,-0.131,0.000,-0.350,-0.450,0.2760,-0.131000,0
"22nd Century Group, Inc. (AMEX:XXII)",-0.6010,-0.8140,-0.9330,-3.5800,-3.050,1.960,-11.300,-11.300,-10.500,-4.457556,...,0.219,0.110,-4.280,-1.020,-1.840,-5.750,-2.030,-5.7700,-2.262333,0


In [22]:
finalData.shape

(2509, 71)

In [23]:
# Randomly splitting data into training and test
np.random.seed(1)
sampleIndex = np.random.randint(0,2,size=finalData.shape[0])

sampleIndex = np.array(sampleIndex,dtype=np.bool)

traning = finalData[~sampleIndex]
test    = finalData[sampleIndex]

In [25]:
# Fit logistic regression
from sklearn import datasets, linear_model

regr = linear_model.LogisticRegression()

regr.fit(traning.iloc[:,:-1],traning.iloc[:,-1])

In [36]:
errorRate = 1 - regr.score(test.iloc[:,:-1],test.iloc[:,-1])
errorRate

0.12183156173344234

In [30]:
# Fit decision tree classifier and set tuning parameter with values provided 
from sklearn import tree
clf = tree.DecisionTreeClassifier(max_depth=8,min_samples_leaf=20,min_impurity_split=0.01)
clf.fit(traning.iloc[:,:-1],traning.iloc[:,-1])

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=8,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=0.01,
            min_samples_leaf=20, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [37]:
errorRate2 = 1 - clf.score(test.iloc[:,:-1],test.iloc[:,-1])
errorRate2

0.12101390024529846